### PENDING:

1. Retain records when player out of game
2. Add: 
    - records of faced-off cards and revise prob of lie
    - when only 2 players remain, 1 player has 1 card remains, the other always bust despite of the style & risk
3. Make modular function for moves
4. Bootstrap
5. Data viz
6. Interactive with real human playing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Set up game:

In [2]:
moves = ["bust", "pass", "true", "lie"]

Set up player

In [3]:
player = np.array(["Thanos", "Ironman", "Batman", "Human"])
print("Number of player must be divided by 4") if len(player) % 4 != 0 else len(player)

4

Set up honesty level for each player. Honesty runs from 0 to 1, with 0 being always lie and 1 being always honest

In [4]:
honesty = np.random.rand(4)
honesty

array([0.49471129, 0.78015494, 0.77661183, 0.01024954])

### Set up style of player:

yolo: This player makes a move simply based on flipping a coin. If the coin tells him to play true while he does not have the card, he will just pass the move

safe: this player will only play true or pass. He nevers bust or lie to avoid the risk of picking up the cards.

thinker: this player analyses all players and keep track of faced-off cards, and consider risks and rewards before making a move

In [5]:
style = np.random.choice(["yolo", "thinker", "safe"], replace = True, size = 4)
style

array(['thinker', 'thinker', 'yolo', 'yolo'], dtype='<U7')

Reward and penalty matrix for current player:

![table1.png](table1.png)

Flow of codes for each game: assemble deck, split cards to each player, play.

### Assign cards for each player

In [9]:
ranks = np.append(np.arange(2,11), ["A", "J", "Q", "K"])
suit_num = 4
deck_num = len(player)/suit_num
deck = np.repeat(ranks, 4*deck_num)
card_num = len(deck)/len(player)
cards = []
for i in range(1,5):
    idx = np.random.choice(range(0,len(deck)), replace = False, size = int(card_num))
    cards.append(np.sort(deck[idx]))
    deck = np.delete(deck, idx)
cards

[array(['2', '2', '4', '5', '7', '7', '9', 'A', 'A', 'A', 'K', 'K', 'Q'],
       dtype='<U11'),
 array(['10', '3', '3', '4', '5', '5', '5', '6', '8', 'A', 'J', 'K', 'Q'],
       dtype='<U11'),
 array(['10', '3', '6', '6', '7', '8', '8', '8', '9', '9', 'J', 'K', 'Q'],
       dtype='<U11'),
 array(['10', '10', '2', '2', '3', '4', '4', '6', '7', '9', 'J', 'J', 'Q'],
       dtype='<U11')]

In [50]:
ranks = np.append(np.arange(2,11), ["A", "J", "Q", "K"])
suit_num = 4
deck_num = len(player)/suit_num
deck = np.repeat(ranks, 4*deck_num)
card_num = len(deck)/len(player)
cards = []
for i in range(1,5):
    idx = np.random.choice(range(0,len(deck)), replace = False, size = int(card_num))
    cards.append(np.sort(deck[idx]))
    deck = np.delete(deck, idx)

player_remain = player
player_num = len(player_remain)
#Initiate round number:
round_num = 1

#Initiate placeholder for winners:
winner = []

#Initiate placehoder for round winner:
round_winner = ""
anchor = None

#Initiate tracking of picked up cards in row of 3+
r3_records = [[],[],[],[]]

#Initiate probability of lying and probability of bust
prob_lie = np.repeat(0.5, player_num)
prob_bust = np.repeat(0.5, player_num)

#Initiate placehoder for moves played
moves_records = np.zeros((4, 4)) 
    #row 1 is times when lie is exposed, row 3 is number of place move 
    #row 3 is times when bust is made, row 4 is number of moves

#Initiate tracking of safe cards played by round starter after each failed bust
safe_num = 0

#Game stops when only 1 player remains:
while player_num > 1:
    #For each round:
    
    #initiate number of 'pass' move and initiate status of 'bust' move ('False' means no 'bust' happens yet)
    pass_num = 0
    bust = False
    
    #Initiate string of placed cards:
    place = []
    
    #Index of players for current round
    player_idx = np.where(np.isin(player, player_remain))[0]
    #First player of round 1 is random, of 2nd round onwards is the previous round's winner:
    first_player = np.random.choice(range(len(player))) if round_num == 1 else np.where(player == round_winner)[0][0]
    print("ROUND ", round_num)
    print("first player:", player[first_player])
    
    #First move of round is always to place a card:
    idx = np.random.choice(range(len(cards[first_player])))    
    place.append(cards[first_player][idx]) 
    cards[first_player] = np.delete(cards[first_player], idx)
    idx = np.random.choice(np.where(ranks != place[-1])[0])
    call = ranks[idx] if (honesty[first_player] < 0.5) else place[-1]
    moves_records[1,first_player] += 1
    moves_records[3,first_player] += 1
    print("place: ", place, "call: ", call)
    
    #Initiate string of players:
    previous_player = first_player
    remain = np.where(~np.isin(player, winner))[0]
    remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])               
    current_player = remain[1]
    next_player = remain[2] if player_num >2 else remain[0]    
    
    #Initiate count of cards cleared:
    cleared_num = [0, 0, 0, 0]
    cleared_num[first_player] += 1
    
    #Round stops when all other players play 'pass' move sequentially or when there is a bust
    while (pass_num < (player_num - 1)) & (bust == False):
        #update probability of lie and bust
        if moves_records[1,previous_player] != 0:
            prob_lie[previous_player] = moves_records[0,previous_player]/moves_records[1,previous_player] 
        if moves_records[3,current_player] != 0:
            prob_bust[current_player] = moves_records[2,current_player]/moves_records[3,current_player]
        #estimated reward and penalty
        r3_num = np.nanmean(r3_records[current_player]) if len(r3_records[current_player]) !=0 else 0
        cleared_num[current_player] += 1          
        
        reward_bust = np.array([r3_num*(1-prob_lie[previous_player]), len(place)*prob_lie[previous_player], 0, 0])
        reward_pass = np.array([0,0, 0, 0])
        reward_true = np.array([0, 0, 
                                (cleared_num[current_player] + safe_num)*prob_bust[next_player] if call in cards[current_player] else 0,
                                (cleared_num[current_player])*(1-prob_bust[next_player]) if call in cards[current_player] else 0])
        reward_lie = np.array([0, 0, 
                               r3_num*prob_bust[next_player], 
                               cleared_num[current_player]*(1-prob_bust[next_player])])
        
        penalty_bust = np.array([len(place)*(1-prob_lie[previous_player]), 0, 0, 0])
        penalty_pass = np.array([0, 0, 0, 0])
        penalty_true = np.array([0, 0, 0, 0])
        penalty_lie = np.array([0, 0, len(place)*prob_bust[next_player], 0])
        
        net_reward = [reward_bust - penalty_bust, reward_pass - penalty_pass,
                      reward_true - penalty_true, reward_lie - penalty_lie]       
        
        #next player play a move:
        if style[current_player] == "yolo":
            move = np.random.choice(range(len(moves)))
            if move == 0: #bust
                bust = True
                moves_records[2,current_player] += 1 #update bust move to record
                if place[-1] == call:
                    cards[current_player] = np.append(cards[current_player], place)  
                else: 
                    if anchor is not None: busted_player = anchor  
                    else: busted_player = previous_player
                    cards[busted_player] = np.append(cards[busted_player], place)
                    moves_records[0,busted_player] += 1 #update lie exposed to record
                temp = pd.Series(place).value_counts()
                r3_records[current_player].append(temp[np.where(temp >= 3)[0]].sum())
                pass_num = 0           
            elif move == 1: #pass
                pass_num += 1
            elif move == 2: #true
                #if the desired card is not available, play 'pass' move:
                if call in cards[current_player]:
                    place.append(call)
                    cards[current_player] = np.delete(cards[current_player], 
                                                      np.where(cards[current_player] == call)[0][0])
                    pass_num = 0
                else:
                    pass_num += 1
                    move = 1
            else:
                #place a 'lie' move
                idx = np.random.choice(np.where(cards[current_player]!=call)[0])                
                place.append(cards[current_player][idx])
                cards[current_player] = np.delete(cards[current_player], idx)
                pass_num = 0
        elif style[current_player] == "thinker":
            move = np.where(net_reward == np.amax(net_reward))[0][0]
            if move == 0:
                bust = True
                moves_records[2,current_player] += 1 #update bust move to record
                if place[-1] == call:
                    cards[current_player] = np.append(cards[current_player], place)  
                else: 
                    if anchor is not None: busted_player = anchor  
                    else: busted_player = previous_player
                    cards[busted_player] = np.append(cards[busted_player], place)
                    moves_records[0,busted_player] += 1 #update lie exposed to record
                temp = pd.Series(place).value_counts()
                r3_records[current_player].append(temp[np.where(temp >= 3)[0]].sum())
                pass_num = 0 
            elif move == 1:
                pass_num += 1
            elif move == 2: #true or pass
                #if the desired card is not available, play 'pass' move:
                if call in cards[current_player]: #true
                    place.append(call)
                    cards[current_player] = np.delete(cards[current_player], 
                                                      np.where(cards[current_player] == call)[0][0])
                    pass_num = 0
                else: #pass
                    pass_num += 1
                    move = 1
            else: #lie
                idx = np.random.choice(np.where(cards[current_player]!=call)[0])                
                place.append(cards[current_player][idx])
                cards[current_player] = np.delete(cards[current_player], idx)
                pass_num = 0
        else: #safe style
            #if the desired card is not available, play 'pass' move:
            if call in cards[current_player]: #true
                place.append(call)
                cards[current_player] = np.delete(cards[current_player], np.where(cards[current_player] == call)[0][0])
                pass_num = 0
                move = 2
            else: #pass
                pass_num += 1
                move = 1
        moves_records[3,current_player] += 1
        print(player[current_player],"-",style[current_player],", plays", moves[move])
        print("place:", place)

        #anchor previous player whose fate is on hold
        if pass_num == 1: anchor = previous_player
        
        #Increment safe cards played by round starters after a failed bust
        if current_player == first_player & bust == False: safe_num += 1       
           
        #count cards of players remain in the round
        card_count = [len(i) for i in cards]

#RECHECK THESE
        remain = np.where(~np.isin(player, winner))[0]
        if (move == 0) & (place[-1] != call):
            remain = np.roll(remain, len(remain)-np.where(remain == busted_player)[0][0])
        elif move == 1:
            remain = np.roll(remain, len(remain)-np.where(remain == anchor)[0][0])
        else:
            remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])        

        #Update the round winner:
        if bust == True:
            idx_win = busted_player if place[-1] == call else current_player
            if len(cards[idx_win]) == 0: idx_win = remain[1]  
            round_winner = player[idx_win]
        if pass_num == player_num - 1:
            if anchor is not None: idx_win = anchor
            if len(cards[idx_win]) == 0: idx_win = remain[1]
            round_winner = player[idx_win]
        
        #if pass_num == player_num - 1:
        #    if current_player - pass_num >=0:
        #        idx_win = current_player - pass_num 
        #    else:
        #        idx_win = current_player - pass_num + player_num
        #    if len(cards[idx_win]) == 0:
        #        idx_win = idx_win + 1 if (idx_win < player_num - 1) else 0
        #    round_winner = player[idx_win]
        print("\t",cards[current_player])
        print("\t player num: ", player_num, ", round winner: ", round_winner)
        print("\t anchor: ", player[anchor] if anchor is not None else "none", 
              ", busted: ", player[busted_player])        
        #remove player with 0 cards out of game
        if anchor is not None:
            if (card_count[anchor] == 0) & (pass_num == player_num - 1): winner.append(player[anchor])
            if (card_count[anchor] == 0) & (move > 1): winner.append(player[anchor])
            if (card_count[anchor] == 0) & (move == 0): winner.append(player[anchor])
            #reset anchor when his fate is no longer on hold
            if (player[anchor] in winner) | (move != 1) | (pass_num == player_num - 1): anchor = None
        if (move > 1) & (card_count[previous_player] == 0): winner.append(player[previous_player])
        if (move == 0) & (card_count[previous_player] == 0): winner.append(player[previous_player])
              
        player_num = len(player) - len(winner)        
        if player_num == 1: bust = True
        
        #increment of the position of player
        previous_player = current_player 
        remain = np.where(~np.isin(player, winner))[0]
        remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])               
        current_player = remain[1] if player_num >1 else previous_player
        next_player = remain[2] if player_num >2 else remain[0]

        print("\t winner: ", winner, ", remain:", player[remain])
    #Update round number
    round_num += 1

ROUND  1
first player: Ironman
place:  ['10'] call:  10
Batman - yolo , plays bust
place: ['10']
	 ['2' '3' '4' '5' '6' '7' 'A' 'J' 'J' 'J' 'K' 'K' 'Q' '10']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Ironman
	 winner:  [] , remain: ['Batman' 'Human' 'Thanos' 'Ironman']
ROUND  2
first player: Ironman
place:  ['K'] call:  K
Batman - yolo , plays true
place: ['K', 'K']
	 ['2' '3' '4' '5' '6' '7' 'A' 'J' 'J' 'J' 'K' 'Q' '10']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Ironman
	 winner:  [] , remain: ['Batman' 'Human' 'Thanos' 'Ironman']
Human - yolo , plays pass
place: ['K', 'K']
	 ['10' '3' '6' '6' '7' '8' '8' '8' '9' 'A' 'A' 'J' 'Q']
	 player num:  4 , round winner:  Ironman
	 anchor:  Batman , busted:  Ironman
	 winner:  [] , remain: ['Human' 'Thanos' 'Ironman' 'Batman']
Thanos - thinker , plays bust
place: ['K', 'K']
	 ['10' '10' '2' '2' '2' '3' '4' '5' '5' '7' '8' '9' 'K' 'K' 'K']
	 player num:  4 , round winner:  Ironman
	 anchor: 

	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Ironman' 'Batman' 'Human' 'Thanos']
ROUND  26
first player: Ironman
place:  ['10'] call:  10
Batman - yolo , plays lie
place: ['10', 'Q']
	 ['5' '6' '7' 'K' 'Q' 'Q' '4' '7' '5']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Batman' 'Human' 'Thanos' 'Ironman']
Human - yolo , plays true
place: ['10', 'Q', '10']
	 ['6' '7' '8' '8' 'A' 'A' '3' '3' '3' 'A' '2']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Human' 'Thanos' 'Ironman' 'Batman']
Thanos - thinker , plays true
place: ['10', 'Q', '10', '10']
	 ['2' '2' '5' '8' 'K' 'K' '8' '2' '7' 'Q' '6' 'K' '4' '5']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Thanos' 'Ironman' 'Batman' 'Human']
Ironman - thinker , plays bust
place: ['10', 'Q', '10', '10']
	 ['4' 'A' '9' '3' '4' '10' '6' 'J' 'J' 'J' '9' 'J' '9' '9' '10' '

Ironman - thinker , plays bust
place: ['K']
	 ['J' 'J' 'J' '10' '10' '10' '2' '6' '2' '2' '9' '9' '9' 'Q' '9']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Ironman' 'Batman' 'Human' 'Thanos']
ROUND  48
first player: Ironman
place:  ['2'] call:  2
Batman - yolo , plays pass
place: ['2']
	 ['5' '6' '5' '3' '5' '6' 'J' 'A']
	 player num:  4 , round winner:  Ironman
	 anchor:  Ironman , busted:  Thanos
	 winner:  [] , remain: ['Batman' 'Human' 'Thanos' 'Ironman']
Human - yolo , plays true
place: ['2', '2']
	 ['8' 'A' 'A' '3' '4' 'Q' '7' '4' '4' '4']
	 player num:  4 , round winner:  Ironman
	 anchor:  Ironman , busted:  Thanos
	 winner:  [] , remain: ['Human' 'Thanos' 'Ironman' 'Batman']
Thanos - thinker , plays lie
place: ['2', '2', '7']
	 ['8' 'K' '8' '6' '3' '7' '3' 'Q' '5' 'A' '10' '7' 'Q' 'K' '8' 'K' 'K']
	 player num:  4 , round winner:  Ironman
	 anchor:  none , busted:  Thanos
	 winner:  [] , remain: ['Thanos' 'Ironman' 'Batm

	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman'] , remain: ['Batman' 'Human' 'Thanos']
ROUND  77
first player: Thanos
place:  ['Q'] call:  9
Batman - yolo , plays true
place: ['Q', '9']
	 ['3' '5' '6' 'A' 'K' 'Q' '3' '10' '10' 'J' 'J' '9' '2' '5' '5' '5']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman'] , remain: ['Batman' 'Human' 'Thanos']
Human - yolo , plays bust
place: ['Q', '9']
	 ['8' 'A' '3' '4' '7' '4' '4' '2' '2' '9' '7' '8' '9' '6' '6' 'Q' 'A' 'J'
 'Q' '9']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
ROUND  78
first player: Thanos
place:  ['K'] call:  7
Batman - yolo , plays lie
place: ['K', '5']
	 ['3' '5' '6' 'A' 'K' 'Q' '3' '10' '10' 'J' 'J' '9' '2' '5' '5']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman'] , remain: ['Batman' 'Human' 'Tha

place: ['A', '6', '2', '6']
	 ['A' '3' '4' '4' '4' 'Q' 'Q' '9' '3' '8' 'A' '6' '2' '6']
	 player num:  3 , round winner:  Batman
	 anchor:  Thanos , busted:  Batman
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
ROUND  97
first player: Batman
place:  ['9'] call:  9
Human - yolo , plays bust
place: ['9']
	 ['A' '3' '4' '4' '4' 'Q' 'Q' '9' '3' '8' 'A' '6' '2' '6' '9']
	 player num:  3 , round winner:  Batman
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
ROUND  98
first player: Batman
place:  ['3'] call:  3
Human - yolo , plays true
place: ['3', '3']
	 ['A' '4' '4' '4' 'Q' 'Q' '9' '3' '8' 'A' '6' '2' '6' '9']
	 player num:  3 , round winner:  Batman
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
Thanos - thinker , plays lie
place: ['3', '3', 'K']
	 ['8' '2' '7' '7' '2' '10' '7' 'Q' '8']
	 player num:  3 , round winner:  Batman
	 anchor:  none , busted:  Batman
	 winner:  ['Ironma

	 anchor:  Thanos , busted:  Thanos
	 winner:  ['Ironman', 'Batman'] , remain: ['Human' 'Thanos']
ROUND  119
first player: Thanos
place:  ['Q'] call:  J
Human - yolo , plays pass
place: ['Q']
	 ['4' '4' '9' '6']
	 player num:  2 , round winner:  Thanos
	 anchor:  Thanos , busted:  Thanos
	 winner:  ['Ironman', 'Batman'] , remain: ['Human' 'Thanos']
ROUND  120
first player: Thanos
place:  ['5'] call:  J
Human - yolo , plays bust
place: ['5']
	 ['4' '4' '9' '6']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman', 'Batman'] , remain: ['Human' 'Thanos']
ROUND  121
first player: Human
place:  ['4'] call:  3
Thanos - thinker , plays lie
place: ['4', '5']
	 []
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Thanos
	 winner:  ['Ironman', 'Batman'] , remain: ['Thanos' 'Human']
Human - yolo , plays pass
place: ['4', '5']
	 ['4' '9' '6']
	 player num:  2 , round winner:  Human
	 anchor:  Thanos , busted:  Thanos
	 winner:  ['Ironman'

In [51]:
player[remain]
remain
player[busted_player]

array(['Human'], dtype='<U7')

array([3], dtype=int64)

'Thanos'

In [52]:
player
player_num
card_count
anchor
winner
player[remain]

array(['Thanos', 'Ironman', 'Batman', 'Human'], dtype='<U7')

1

[0, 0, 0, 3]

['Ironman', 'Batman', 'Thanos']

array(['Human'], dtype='<U7')